# **Sudoku Solver**

**Group members:** Ju Won Kim, Matt Coley, Avni Gulrajani, Sue Kim, Aarambh Sanoria

Sudoku is a logic-based combinatorial number-placement puzzle. The code implements two algorithms to solve Sudoku puzzles:

1. **AC-3 Algorithm:** This algorithm enforces arc consistency and propagates constraints to reduce the domain size of variables by ensuring the consistency of assignments. It iteratively removes values from the domain of variables based on constraints until a solution is found or it determines that no solution exists.
2. **Backtracking with Minimum Remaining Values (MRV) Heuristic:** This algorithm recursively searches for a solution by assigning values to empty cells and backtracking when a solution is not valid. It selects the next cell to be assigned a value based on the MRV heuristic, which prioritizes cells with the fewest remaining valid values.
We have included a set of predefined Sudoku puzzles stored as lists of lists. The main block allows the user to choose a puzzle to solve interactively by inputting a number between 1 and 5. You can try the puzzle [here](https://colab.research.google.com/drive/1NVO3W0-mAX2bLB0z7dQzW-2prCUO3E9j?usp=sharing)

[GitHub Repository](https://github.com/Armdoor/SudokuSolver)

In [21]:
import csv
import copy

#######################################
# ------------  CSP class  ---------- #
#######################################

# Class for representing the CSP
# Each cell is a variable with a domain of numbers 1 through 9, unless it's fixed
# Constraints are the rules of sudoku
class CSP:
    def __init__(self, puzzle):

        self.domains = {}
        self.arcs = {}

        for row in range(9):
            for col in range(9):
                cell = (row, col)

                # Set the domain for each variable. Fixed cells only have one number in their domain.
                self.domains[cell] = {puzzle[row][col]} if puzzle[row][col] != 0 else set(range(1,10))

                # # Setting up the constraints by creating arcs for each cell in the grid
                self.build_arcs(cell)

    # Creates the cell's arcs
    def build_arcs(self, cell):
        self.arcs[cell] = []
        # Constraints for cells in the same column
        for row in range(9):
            if row != cell[0]:
                self.arcs[cell].append((row, cell[1]))

        # Constraints for cells in the same row
        for col in range(9):
            if col != cell[1]:
                self.arcs[cell].append((cell[0], col))

        # Constraints for cells in the same subgrid
        sub_row = cell[0] // 3 * 3
        sub_col = cell[1] // 3 * 3

        for row in range(sub_row, sub_row + 3):
            for col in range(sub_col, sub_col + 3):
                if row != cell[0] or col != cell[1]:
                    self.arcs[cell].append((row, col))

    # Converts the CSP back to a list of lists
    def get_puzzle(self):

        puzzle = [[0] * 9 for _ in range(9)]

        for cell, domain in self.domains.items():
            # If the puzzle is solved, each domain should contain exactly one value
            if len(domain) == 1:
                puzzle[cell[0]][cell[1]] = (next(iter(domain)))
            else:
                # If any cell still has multiple values, it means the puzzle isn't fully solved
                puzzle[cell[0]][cell[1]] = 0
        return puzzle


    #######################################
    # ---- Arc consistency functions ---- #
    #######################################

    # Following the approach in:
    # Luna, David. “Programming a Sudoku solver” davidlunadeleon, 2021,
    # https://www.davidlunadeleon.com/en/blog/sudoku_csp.

    # Creating a queue of pairs (x,y) of neighboring cells to begin arc consistency checking
    def ac3(self):
        queue = [(cell, neighbor) for cell in self.domains for neighbor in self.arcs[cell]]
        self.check(queue)

    # If inconsistencies are found, then (neighbor, x) is inserted in the queue
    # Ends when all relevant pairs have been checked
    # Returns True if no new conflicts are discovered and False otherwise
    def check(self, queue):
        while queue:
            cell1, cell2 = queue.pop(0)
            if self.remove_inconsistencies(cell1, cell2):
                for neighbor in self.arcs[cell1]:
                    queue.append((neighbor, cell1))

        for neighbor in self.arcs[cell1]:
            if len(self.domains[neighbor]) < 1:
                return False
        return True

    # For every (x, y) pair, make sure there is a suitable value in y's domain for each value in x's domain.
    # Remove a value from x if there's no suitable option in y's domain
    def remove_inconsistencies(self, cell1, cell2):
        if len(self.domains[cell2]) == 1:
            number = next(iter(self.domains[cell2]))

            # If there's only one value in y's domain, and that value is also in x's domain,
            # remove the value from x's domain.
            if number in self.domains[cell1]:
                self.domains[cell1].remove(number)
                return True
        return False


    #######################################
    # -------- Solving functions -------- #
    #######################################

    # Checking whether the CSP is solved
    # Solved if every variable has been assigned a value (has only one value in its domain)
    def is_solved(self):
        return all(len(domain) == 1 for domain in self.domains.values())

    # Employing the MRV heuristic.
    # Finding the cell with the minimum remaining values in its domain
    # Only considering cells with more than one value remaining (not currently assigned)
    def mrv(self):
        cell = None
        min_domain_size = float('inf')

        for key in self.domains:
            domain_size = len(self.domains[key])
            if domain_size > 1 and domain_size < min_domain_size:
                cell = key
                min_domain_size = domain_size

        return cell


    # Solves the puzzle using backtracking, MRV, and AC3
    def solve(self):

        # First, check if the puzzle is already solved. If so, no further work necessary.
        if self.is_solved():
            return True

        # Selecting a cell to assign a value to, based on the MRV heuristic
        cell = self.mrv()

        # Trying each value in the cell's domain until we find one that works
        current_domain = list(self.domains[cell])
        for number in current_domain:

            # Maintaining a backup of the original domains, in case we need to backtrack
            original_domains = copy.deepcopy(self.domains)

            # Assigning a potential value to a cell by removing other values in its domain
            self.domains[cell] = {number}

            # Enforcing arc consistency after the assignment
            # Moving on to the rest of the puzzle if a potential solution still exists
            queue = [(neighbor, cell) for neighbor in self.arcs[cell]]
            if self.check(queue):
                if self.solve():
                    return True

            # Backtracking if the current value doesn't succeed
            self.domains = original_domains

        # If no valid solution exists at the current point
        return False


######################################
# --------- Other functions -------- #
######################################

# Converts the csv file to a list of lists representing the puzzle
def read_puzzle(file_path):

    puzzle = []
    with open(file_path) as file:
        reader = csv.reader(file)
        for row in reader:
            puzzle.append([int(number) for number in row])
    return puzzle

# Prints the puzzle neatly
def print_puzzle(puzzle):

    # 9 is the puzzle size
    for row in range(9):
        if row % 3 == 0 and row != 0:
            print("-" * 21)
        for col in range(9):
            if col % 3 == 0 and col != 0:
                print("|", end=" ")

            if col == 8:
                print(puzzle[row][col])
            else:
                print(puzzle[row][col], end=" ")

# Putting together all components of the CSP
# Set up CSP from list, use AC3, solve, convert back to list (if solvable)
def solve_puzzle(puzzle):
    csp = CSP(puzzle)
    csp.ac3()
    if csp.solve():
        return csp.get_puzzle()
    else:
        return None

######################################
# ---------- Main function --------- #
######################################

def main():
    while True:
      puzz_num = input("Please enter a number between 1 and 100, or type 'quit' to exit: ")

      if puzz_num.lower() == 'quit':
          print("Goodbye. ")
          break

      if puzz_num.isdigit():  # Check if input is a digit
          puzz_num = int(puzz_num)
          if 1 <= puzz_num <= 100:
              puzzle = read_puzzle(f"/content/drive/MyDrive/CMSC421 - Final Project/puzzles/puzzle_{puzz_num}.csv")
              print("Unsolved puzzle:")
              print_puzzle(puzzle)
              if input("Show solution? (enter yes or no): ").lower() == "yes":
                  puzzle = solve_puzzle(puzzle)
                  if puzzle:
                      print("Solved puzzle:")
                      print_puzzle(puzzle)
                      if input("Check if solution is valid? (enter yes or no): ").lower() == "yes":
                          if isValidSudoku(puzzle):
                              print("Solution is valid!")
                          else:
                              print("Solution is invalid!")
                  else:
                      print("No solution found")
          else:
              print("Entered an invalid number. Please enter a number between 1 and 100.")
      else:
          print("Invalid input. Please enter a valid number or type 'quit' to exit.")


if __name__ == "__main__":
    main()

Please enter a number between 1 and 100, or type 'quit' to exit: 1
Unsolved puzzle:
0 3 0 | 6 0 4 | 2 5 0
2 0 0 | 3 0 0 | 1 0 0
0 0 9 | 2 5 0 | 0 0 8
---------------------
0 0 0 | 7 0 0 | 0 9 0
9 0 2 | 0 0 5 | 0 3 1
7 4 0 | 1 0 0 | 0 0 6
---------------------
0 9 0 | 0 7 0 | 6 0 4
0 0 1 | 0 6 0 | 0 0 0
5 2 6 | 8 0 0 | 9 0 3
Show solution? (enter yes or no): yes
Solved puzzle:
8 3 7 | 6 1 4 | 2 5 9
2 5 4 | 3 9 8 | 1 6 7
6 1 9 | 2 5 7 | 3 4 8
---------------------
1 8 3 | 7 2 6 | 4 9 5
9 6 2 | 4 8 5 | 7 3 1
7 4 5 | 1 3 9 | 8 2 6
---------------------
3 9 8 | 5 7 2 | 6 1 4
4 7 1 | 9 6 3 | 5 8 2
5 2 6 | 8 4 1 | 9 7 3
Check if solution is valid? (enter yes or no): yes
Solution is valid!
Please enter a number between 1 and 100, or type 'quit' to exit: q
Invalid input. Please enter a valid number or type 'quit' to exit.
Please enter a number between 1 and 100, or type 'quit' to exit: quit
Goodbye. 


In [ ]:
##----Validates the solution(Optional)----#
def isinRange(puzzle):
  N = 9
  for i in range(0, N):
    for j in range(0, N):
      if ((puzzle[i][j] <= 0) or
          (puzzle[i][j] > 9)):
        return False
  return True


def isValidSudoku(puzzle):
  N = 9
  if (isinRange(puzzle) == False):
    return False
  unique = [False] * (N + 1)
  for i in range(0, N):
    for m in range(0, N + 1):
      unique[m] = False
    for j in range(0, N):
      Z = puzzle[i][j]
      if (unique[Z] == True):
        return False
      unique[Z] = True

  # Traverse each column
  for i in range(0, N):
    for m in range(0, N + 1):
      unique[m] = False
    for j in range(0, N):
      Z = puzzle[j][i]
      if (unique[Z] == True):
        return False

      unique[Z] = True

  # Traverse each block of size 3 * 3 in puzzle[][] array
  for i in range(0, N - 2, 3):
    for j in range(0, N - 2, 3):
      for m in range(0, N + 1):
        unique[m] = False
      for k in range(0, 3):
        for l in range(0, 3):
          X = i + k
          Y = j + l
          Z = puzzle[X][Y]
          if (unique[Z] == True):
            return False
          unique[Z] = True

  return True